In [ ]:
from autumn.core.project import get_project
from autumn.infrastructure.remote.aws import springboard
from summer2.extras.test_models import sirs_parametric_age

In [ ]:
def get_model():
    m = sirs_parametric_age()
    return m

In [ ]:
# Simple example function - first argument must be the TaskBridge
# use bridge.out_path as the destination for storing files,
# and bridge.logger for any logging you wish to have on S3

def full_task(bridge: springboard.TaskBridge):
    logger = bridge.logger
    out_path = bridge.out_path
    m = get_model()
    m.run()
    df = m.get_derived_outputs_df()
    df.to_csv(out_path / "derived_outputs.csv")

In [ ]:
# Request a machine
# arguments are min_cores, min_ram (gb), and machine type (one of "compute", "general" or "memory")
# for calibration jobs this should always be compute

mspec = springboard.EC2MachineSpec(2,1,"compute")

In [ ]:
# Create a TaskSpec
# If your task function takes further arguments, include them as a dictionary here
# eg
# def mytask(bridge, x, y):
# ...
# TaskSpec(mytask, {"x": x_value, "y": y_value})

tspec = springboard.TaskSpec(full_task)

In [ ]:
# Generate a run_path (run_id) of the format projects/APPLICATION/REGION/run_description

run_path = springboard.get_autumn_project_run_path("sirs_parametric_age", "global", "demo_task")

In [ ]:
run_path

In [ ]:
s3task, sshrunner, ios = springboard.launch_synced_autumn_task(tspec, mspec, run_path, "springboard")

In [ ]:
s3task.get_status()

In [ ]:
from time import sleep

In [ ]:
def wait_task(s3task):
    status = ""
    while status not in ["SUCCESS", "FAILED"]:
        status = s3task.get_status()
        print(status)
        sleep(10)
    return status

In [ ]:
status = wait_task(s3task)

In [ ]:
from autumn.core.runs import ManagedRun

In [ ]:
mr = ManagedRun(run_path)

In [ ]:
mr.remote.list_contents()

In [ ]:
for f in mr.remote.list_contents():
    mr.remote.download(f)

In [ ]:
with open(mr.local_path / "log/task.log") as f:
    print(f.read())

In [ ]:
# If the task has failed, then read the crash log
if status == "FAILED":
    with open(mr.local_path / "log/crash.log") as f:
        print(f.read())

In [ ]:
mr.local_path

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(mr.local_path / "output/derived_outputs.csv")

df